In [1]:
import csv
import json
import struct
from functools import reduce

# trans .csv to .param

In [50]:
def trans_csv_to_param():
    with open('mynet.param','w') as p: 
        with open('mynet.csv','r') as f:
            f_csv = csv.reader(f)

            blobs = []
            outputcontent = ''
            row_index = 0
            for row in f_csv:   ## read rows in csv
                # normal lines <layertype   layername   inputNum outputNum inputBlob outputBlob params>
                col_index = 0
                for col in row:
                    if col_index == 0:                # layertype
                        outputcontent += '%-16s ' % (col)
                    elif col_index == 1:              # layername
                        outputcontent += '%-32s ' % (col)
                    elif col_index == 2:              # inputNum
                        inputBlobNum = int(col)
                        outputcontent += ' %s' % (col)
                    elif col_index == 3:              # outputNum
                        outputBlobNum = int(col)
                        outputcontent += ' %s' % (col)
                    elif col_index > 3 and col_index <= 3+inputBlobNum+outputBlobNum:  # inputBlob outputBlob
                        blobs.append(col)
                        outputcontent += ' %s' % (col)
                    else:                             # params
                        outputcontent += ' %s' % (col)
                    col_index += 1
                outputcontent += '\n'
                row_index += 1  ## end of read rows in csv
            outputcontent = '7767517\n'+'%s '%(row_index)+'%s\n'%(len(list(set(blobs))))+outputcontent
            p.write(outputcontent)

# process conv unit from json to csv
# save json buffer data to bin

### save memorydata [1]

In [43]:
def init_bin():
    a = [1]
    with open('mynet.bin','wb') as f:
        for i in a:
            print(struct.pack('<f',i))
            f.write(struct.pack('<f',i))

### import json file

In [2]:
jsonpath = "/home/gx/myproj/flatbuffers/im14_without_q_att.json"
with open(jsonpath,'r') as f_json:
    model = json.load(f_json)

### append new row to csv file

In [3]:
def append_row_to_csv(appendList):
    with open('mynet.csv','a', newline = "") as f:
        csv_writer = csv.writer(f, dialect = "excel")
        csv_writer.writerow(appendList)

### save TF-buffer data to NCNN-bin file 

In [4]:
# FROM o-h-w-i(tflite) TO o-i-h-w(ncnn)
def fix_weight_order(list):
    pass

In [31]:
#save weight & bias data from tflite-json file to ncnn-bin file
def save_data_to_bin(bufferIndex, savefilter=True, append=True):
    bufferData = model['buffers'][bufferIndex]['data']
    flag = 'ab' if append else 'wb'
    with open('mynet.bin',flag) as f:
        if savefilter:
            i = 0x00000000   # flag-structure: weight data need to save as float type
            f.write(struct.pack('<I',i))
            fix_weight_order(bufferData)
        for i in bufferData: # sava bufferdata
            f.write(struct.pack('B',i))
    return

### save several type of layers

In [33]:
def save_conv(name, indexTfLite, inputBlob, outputBlob):
    
    op = model['subgraphs'][0]['operators']
    tensor = model['subgraphs'][0]['tensors']
    
    filterTensor = op[indexTfLite]['inputs'][1]
    biasTensor = op[indexTfLite]['inputs'][2]

    if 'dilation_w_factor' in op[indexTfLite]['builtin_options'].keys():
        dilation_w_factor = op[indexTfLite]['builtin_options']['dilation_w_factor']
    else:
        dilation_w_factor = '1'
        
    if 'stride_w' in op[indexTfLite]['builtin_options'].keys():
        stride_w = op[indexTfLite]['builtin_options']['stride_w']
    else:
        stride_w = '1'

    param = ['0=' + str(tensor[filterTensor]['shape'][0]), # num_output
             '1=' + str(tensor[filterTensor]['shape'][1]), # kernel_w   （kernel_h default）
             '2=' + str(dilation_w_factor),                # rate
             '3=' + str(stride_w),                         # stride
             '4=-233',                                     # same
             '5=1',                                        # has bias
             '6=' + str(reduce(lambda x,y:x*y,tensor[filterTensor]['shape'])) # weight
            ]
    
    append_row_to_csv(['Convolution']+[name]+['1','1']+[inputBlob,name]+param)
    save_data_to_bin(tensor[filterTensor]['buffer'],True)  # save filter's buffer (weight)
    save_data_to_bin(tensor[biasTensor]['buffer'],False)   # save bias's   buffer 
    return outputBlob

def save_slice(name, inputBlob, outputBlob1, outputBlob2):
    append_row_to_csv(['Slice']+[name]+['1','2']+[inputBlob,outputBlob1,outputBlob2]+['0=-233','1=2'])
    return outputBlob1, outputBlob2

def save_interp(name, inputBlob, outputBlob):                         # ResizeNearestNeighbor 2*
    append_row_to_csv(['Interp']+[name]+['1','1']+[inputBlob,outputBlob]+['0=0','1=2','2=2']) 
    return outputBlob

def save_elu(name, inputBlob, outputBlob):
    append_row_to_csv(['Elu']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob
    
def save_relu(name, inputBlob, outputBlob):
    append_row_to_csv(['Relu']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob
    
def save_tanh(name, inputBlob, outputBlob):
    append_row_to_csv(['TanH']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob
    
def save_sigmoid(name, inputBlob, outputBlob):
    append_row_to_csv(['Sigmoid']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob

def save_mul(name, inputBlob1, inputBlob2, outputBlob):
    append_row_to_csv(['BinaryOp']+[name]+['2','1']+[inputBlob1,inputBlob2,outputBlob]+['0=2'])
    return outputBlob


### save 3 types of conv2d unit

In [41]:
def save_conv_unit(unitType, indexTfLite, inputBlob, deconv,  stage=''):
    
    def N(name):
        return stage+'/'+name+'_'+str(indexTfLite)
    
    x = inputBlob
    
    if deconv:
        x = save_interp(('interp'), x, N('interp'))
        
    x = save_conv(N('conv2d'),indexTfLite, x, N('conv2d'))
    
    if unitType == 'TanH':
        x = save_tanh(N('tanh'),x,N('tanh'))
    else:  
        x,y = save_slice(N('slice'), x, N('conv2d')+'_A', N('conv2d')+'_B')

        if unitType == 'Elu':
            x = save_elu(N('elu'),x,N('elu'))
        if unitType == 'Relu':
            x = save_relu(N('relu'),x,N('relu'))

        y = save_sigmoid(N('sigmoid'),y,N('sigmoid'))
        x = save_mul(N('mul'),x,y,N('mul'))   

    return x

In [45]:
save_conv_unit('Elu',13,'concat',True,stage='stage1')

'stage1/mul_13'

In [52]:
init_bin()
stage1 = [13,20,25,30,35, 40,45,50,55,60, 65,70,76,81,87, 92,97]
index = 1
x = 'concat'
for i in stage1:
    unitType = 'TanH' if index == 17 else 'Elu'
    deconv = True if index in [13,15] else False
    x = save_conv_unit(unitType,i,x,deconv,'stage1')
    index += 1

b'\x00\x00\x80?'


In [53]:
trans_csv_to_param()